***Data Preprocessing***


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import pickle

In [7]:
from tensorflow import keras
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.applications import ImageDataGenerator
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [8]:
# Define dataset path
DATASET_PATH = "data"  # Change this to your dataset folder


In [9]:
# Define categories (same as your folder names)
CATEGORIES = ["Blight", "Common_Rust", "Gray_Leaf_Spot", "Healthy"]



In [ ]:
# Image size for CNNs like ResNet
IMG_SIZE = 224

In [10]:
# Store images and labels
data = []
labels = []


In [11]:
import os
print(os.listdir("data"))


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data'

In [ ]:
# Load images from each category

for category in CATEGORIES:
    folder_path = os.path.join(DATASET_PATH, category)
    class_index = CATEGORIES.index(category)  # Assign a label (0,1,2,3)

    for img_name in tqdm(os.listdir(folder_path), desc=f"Processing {category}"):
        img_path = os.path.join(folder_path, img_name)
        try:
            # Read and resize image
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize
            
            # Normalize pixels (0 to 1 range)
            img = img / 255.0

            # Append to dataset
            data.append(img)
            labels.append(class_index)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")


In [ ]:
# Convert lists to numpy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
# Split into Train (70%), Validation (20%), and Test (10%)
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, stratify=labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, stratify=y_temp, random_state=42)

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Data Augmentation for Training Data
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2]
)

In [ ]:
# ✅ Save all data in a Pickle file
with open("data.pkl", "wb") as f:
    pickle.dump({
        "X_train": X_train, "y_train": y_train,
        "X_val": X_val, "y_val": y_val,
        "X_test": X_test, "y_test": y_test
    }, f)

print("✅ Preprocessing Completed & Data Saved in data.pkl!")

***Data model training***


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from sklearn.metrics import classification_report, confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Load data
with open("data.pkl", "rb") as f:
    data = pickle.load(f)
    
X_train, y_train = data["X_train"], data["y_train"]
X_val, y_val = data["X_val"], data["y_val"]
X_test, y_test = data["X_test"], data["y_test"]


In [ ]:
# Load base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base

In [ ]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True),
    CSVLogger("training_log.csv")
]

In [ ]:
# Train model
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=callbacks)


In [ ]:
# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {test_acc:.4f}")

In [ ]:
# Predict and show classification report
y_pred = model.predict(X_test).argmax(axis=1)
print(classification_report(y_test, y_pred, target_names=["Blight", "Common_Rust", "Gray_Leaf_Spot", "Healthy"]))

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CATEGORIES, yticklabels=CATEGORIES)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()